In [1]:
# 11_Hourly_to_HourlyGPKG.ipynb
# Input: hourly CSV (Date, Time, location_name, latitude, longitude, pm25)
# Output: one GPKG per day with a single layer:
#   - hourly_points (all hourly rows for that date, with geometry)

In [2]:
import os
import pandas as pd
import geopandas as gpd

# -------- EDIT THESE --------
HOURLY_CSV = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"
OUT_HOURLY_GPKG_DIR = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\HourlyGPKG"
os.makedirs(OUT_HOURLY_GPKG_DIR, exist_ok=True)
TIMEZONE = "Asia/Manila"
# ----------------------------

# Column names from your sample
DATE_COL     = "Date"
TIME_COL     = "Time"
STATION_COL  = "location_name"
LAT_COL      = "latitude"
LON_COL      = "longitude"
PM_COL       = "pm25"

# Read hourly CSV
df = pd.read_csv(HOURLY_CSV)

# Clean PM and time
df[PM_COL] = pd.to_numeric(df[PM_COL], errors="coerce").clip(lower=0, upper=500)
ts = pd.to_datetime(df[DATE_COL].astype(str) + " " + df[TIME_COL].astype(str), errors="coerce")

# Localize to Asia/Manila (treat as local)
if ts.dt.tz is None:
    ts = ts.dt.tz_localize(TIMEZONE)
else:
    ts = ts.dt.tz_convert(TIMEZONE)

df["ts_local"] = ts
df["date_key"] = df["ts_local"].dt.date
df["hour"] = df["ts_local"].dt.hour

# Remove rows with bad coords or times
df[LAT_COL] = pd.to_numeric(df[LAT_COL], errors="coerce")
df[LON_COL] = pd.to_numeric(df[LON_COL], errors="coerce")
df = df.dropna(subset=["ts_local", PM_COL, LAT_COL, LON_COL]).copy()

# Make sure station names are strings
df[STATION_COL] = df[STATION_COL].astype(str)

WGS84 = "EPSG:4326"
written = 0

for day, df_day in df.groupby("date_key", sort=True):
    # Hourly rows for this day
    hday = df_day[[STATION_COL, "ts_local", "hour", PM_COL, LAT_COL, LON_COL]].copy()
    hday["timestamp"] = hday["ts_local"].dt.strftime("%Y-%m-%d %H:%M:%S%z")

    gdf_hourly = gpd.GeoDataFrame(
        hday.drop(columns=[LAT_COL, LON_COL]),
        geometry=gpd.points_from_xy(hday[LON_COL], hday[LAT_COL]),
        crs=WGS84
    )
    gdf_hourly["date"] = pd.to_datetime(day).strftime("%Y-%m-%d")

    out = os.path.join(OUT_HOURLY_GPKG_DIR, f"hourly_{pd.to_datetime(day).strftime('%Y-%m-%d')}.gpkg")
    if os.path.exists(out):
        os.remove(out)

    gdf_hourly.to_file(out, layer="hourly_points", driver="GPKG")
    written += 1
    print(f"✓ Wrote {os.path.basename(out)} | hourly rows: {gdf_hourly.shape[0]}")

print(f"\nDone. Wrote {written} hourly GPKGs to:\n  {OUT_HOURLY_GPKG_DIR}")


✓ Wrote hourly_2025-01-01.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-02.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-03.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-04.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-05.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-06.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-07.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-08.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-09.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-10.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-11.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-12.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-13.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-14.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-15.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-16.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-17.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-18.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-19.gpkg | hourly rows: 24
✓ Wrote hourly_2025-01-20.gpkg | hourly rows: 24
✓ Wrote hourly_2025-